In [22]:
import sys
import os
import cv2
import numpy as np
import glob
import matplotlib.pyplot as plt
from statistics import mode
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.metrics import f1_score

In [2]:
def load_data(arg, testOrtrain):
    files = glob.glob("./Data/" + testOrtrain + " set/" + arg + "/*")
    img_arr = []
    for f in files:
        img_arr.append(cv2.imread(f))
        
    return np.array(img_arr)

### Colour Feature Extractor

In [38]:
def FeatureGetColours(imgs):
    g_lowerb = (35,0,0) #green
    g_upperb = (90,255,255)
    
    b_lowerb = (90,0,0) #blue
    b_upperb = (135,255,255)
    
    br_lowerb = (0,0,0) #red/brown
    br_upperb = (35,255,255)
    
    num_pixel = np.prod(imgs[0][:,:,0].size)
    f = []
    
    g, bl, br = [],[],[]
    for img in imgs:
        dst = img
        dst = cv2.cvtColor(dst, cv2.COLOR_BGR2HSV)
        green = cv2.inRange(dst, g_lowerb, g_upperb)
        blue = cv2.inRange(dst, b_lowerb, b_upperb)
        brown = cv2.inRange(dst, br_lowerb, br_upperb)
        
        green_pixel = cv2.countNonZero(green)
        blue_pixel = cv2.countNonZero(blue)
        brown_pixel = cv2.countNonZero(brown)
        
        k = [0]*3
        k[0] = 1 if (green_pixel/num_pixel > 0.3) else 0
        k[1] = 1 if (blue_pixel/num_pixel > 0.3) else 0
        k[2] = 1 if (brown_pixel/num_pixel > 0.3) else 0

        g.append(green_pixel/num_pixel)
        bl.append(blue_pixel/num_pixel)
        br.append(brown_pixel/num_pixel)
    
    return g,bl,br

### Dominant Line Detector

In [4]:
def FeatureHaveDominantLine(imgs):
    minLineLength = 40
    maxLineGap = 10
    f = []
    for index,img in enumerate(imgs):
        dst = cv2.bilateralFilter(img,9, 75, 75, cv2.BORDER_DEFAULT)
        gray = cv2.cvtColor(dst,cv2.COLOR_BGR2GRAY)
        edges = cv2.Canny(gray,80,200,apertureSize = 3)
        lines = cv2.HoughLinesP(edges,1,np.pi/180,100,minLineLength,maxLineGap)

        k=0
        if lines is not None and lines.shape[0] > 2:
            k=1
            
#             img2 = img.copy()
#             for i in range(0, len(lines)):
#                 l= lines[i][0]
#                 cv2.line(img2, (l[0], l[1]), (l[2], l[3]), (0,0,255), 3, 8)
#             name = "test/houghlines" + str(index) + ".jpg"
#             cv2.imwrite(name,img2)
            
        f.append(k)
    
    return f

### Angles of lines

In [5]:
def cal_angle(arr):
#     return arr[1]
    if arr[1] == arr[3]:
        return np.pi/2
    if arr[0] == arr[2]:
        return 0
    
    if arr[1] > arr[3]:
        p1 = [arr[0], arr[1]]
        p2 = [arr[2], arr[3]]
    else:
        p1 = [arr[2], arr[3]]
        p2 = [arr[0], arr[1]]
    
    dy = p2[1]-p1[1]
    dx = p2[0]-p1[0]
    return np.arctan(dy/dx)


def FeatureHaveHorizontalVerticalDiagLine(imgs):
    minLineLength = 40
    maxLineGap = 10
    f = []
    h,v,d =[],[],[]
    for index, img in enumerate(imgs):
        dst = cv2.bilateralFilter(img,9, 75, 75, cv2.BORDER_DEFAULT)
        gray = cv2.cvtColor(dst,cv2.COLOR_BGR2GRAY)
        edges = cv2.Canny(gray,80,200,apertureSize = 3)
        lines = cv2.HoughLinesP(edges,1,np.pi/180,60,minLineLength,maxLineGap)
        a = []
        k = img.copy()
        if lines is not None:
            for i in range(0, len(lines)):
                l= lines[i][0]
                a.append(cal_angle(l))
#                 cv2.line(k, (l[0], l[1]), (l[2], l[3]), (0,0,255), 3, 8)
#                 cv2.line(k, Point(lines[i][0], lines[i][1]),Point( lines[i][2], lines[i][3]), Scalar(0,0,255), 3, 8 );
            
#             name = "test/houghlines" + str(index) + ".jpg"
#             cv2.imwrite(name,k)
            a = np.array(a)
            
            vertical = np.count_nonzero(np.logical_and(a > -0.61, a < 0.61))/a.shape[0]
            horizontal = np.count_nonzero(np.logical_and(a >= 1.22, a < 1.92))/a.shape[0]
            diag = np.logical_or(np.logical_and(a >= 0.69, a < 0.87),np.logical_and(a >= 2.26893, a < 2.44346))
            diagonal = np.count_nonzero(diag)/a.shape[0]
            
            h.append(horizontal)
            v.append(vertical)
            d.append(diagonal)
            
        else:
            h.append(0)
            v.append(0)
            d.append(0)
        
        
    return h,v,d
        

### Preparing Data

In [6]:
def prepareInput(extracted_data):
    y_labels = [1,2,3,4]
    x,y = [],[]
    f = [None]*7
    num_feature = 7
    
    for i in range(4):
        f[0],f[1],f[2] = FeatureGetColours(extracted_data[i])
        f[3] = FeatureHaveDominantLine(extracted_data[i])
        f[4],f[5],f[6] = FeatureHaveHorizontalVerticalDiagLine(extracted_data[i])
        
        k = np.column_stack([f[0],f[1],f[2],f[3],f[4],f[5],f[6]])

        x.append(k)
        y.append(np.tile(i,(k.shape[0],1)))

    x_train = np.vstack((x[0],x[1],x[2],x[3]))
    y_train = np.vstack((y[0],y[1],y[2],y[3]))
    x_train = np.asarray(x_train, np.float32)
    y_train = np.asarray(y_train, np.int32)

    return x_train,y_train

In [7]:
def prepareInput_comb(extracted_data): # only the first four features
    y_labels = [1,2,3,4]
    x,y = [],[]
    f = [None]*7
    num_feature = 7
    
    for i in range(4):
        f[0],f[1],f[2] = FeatureGetColours(extracted_data[i])
        f[3] = FeatureHaveDominantLine(extracted_data[i])
        
        k = np.column_stack([f[0],f[1],f[2],f[3]])

        x.append(k)
        y.append(np.tile(i,(k.shape[0],1)))

    x_train = np.vstack((x[0],x[1],x[2],x[3]))
    y_train = np.vstack((y[0],y[1],y[2],y[3]))
    x_train = np.asarray(x_train, np.float32)
    y_train = np.asarray(y_train, np.int32)
    return x_train,y_train

In [8]:
def prepareInput_single(extracted_data, feature):
    y_labels = [1,2,3,4]

    x,y = [],[]
    f = [None]*7
    for i in range(4): #number of catergories
        f[0],f[1],f[2] = FeatureGetColours(extracted_data[i])
        f[3] = FeatureHaveDominantLine(extracted_data[i])
        f[4],f[5],f[6] = FeatureHaveHorizontalVerticalDiagLine(extracted_data[i])
        
        x.extend(f[feature])
        y.append(np.tile(i,(len(f[0]),1)))

    y_train = np.vstack((y[0],y[1],y[2],y[3]))
    x_train = np.asarray(x, np.float32)
    y_train = np.asarray(y_train, np.int32)
    return x_train,y_train

# prepareInput_single(extracted_data, 0)

### Training the SVM 

In [9]:
def trainModel(x_train,y_train, kernel = "LINEAR"):
    svm = cv2.ml.SVM_create()
    svm.setType(cv2.ml.SVM_C_SVC)
    
    k = cv2.ml.SVM_LINEAR
    if (kernel == "RBF"):
        k = cv2.ml.SVM_RBF
    elif (kernel == "POLY"):
        k = 1
        svm.setDegree(2.0)

    svm.setKernel(k)
    
    svm.setTermCriteria((cv2.TERM_CRITERIA_MAX_ITER, 100, 1e-6))
    svm.train(x_train, cv2.ml.ROW_SAMPLE, y_train)
    return svm

### Performance

In [10]:
def getAccuracy(x_train, y_train, model):
    _, response = model.predict(x_train)
    f1 = f1_score(y_train, response, average='macro')
    count = np.equal(response,y_train)
    a = np.count_nonzero(count)/len(y_train)
    print("Accuracy", a)
    print("F1", f1)

### Multi-modal model (all features)

In [62]:
catergories = ["bridge","coast","mountain","rainforest"]
extracted_data = []
testing_data = []
for c in catergories:
    extracted_data.append(load_data(c,"Training"))
    testing_data.append(load_data(c,"Testing"))

x_train,y_train = prepareInput(extracted_data)
x_test,y_test = prepareInput(testing_data)

In [ ]:
f0 = x_train[:,0]
f1 = x_train[:,5]

fig,ax = plt.subplots(figsize=(6,6))
for cat in range(4):
    catergories = ["bridge","coast","mountain","rainforest"]
    i = (y_train == cat)
    ax.scatter(f0[i[:,0]], f1[i[:,0]], label=catergories[cat])

# ax.set_title("PCA of the 1st and 2nd Component")
ax.set_xlabel("Feature: Detected Number of Green Pixels / Total Number of Pixels")
ax.set_ylabel("Feature: Detected Number of Horizontal Line / Total Number of Lines")
legend = ax.legend(loc='upper left', bbox_to_anchor=(1.1, 1))

# fig.savefig("linear-data.jpeg", bbox_inches='tight')

In [ ]:
svm_multi = trainModel(x_train,y_train, "LINEAR")
getAccuracy(x_train, y_train, svm_multi) 
getAccuracy(x_test, y_test, svm_multi) 

In [ ]:
svm_multi = trainModel(x_train,y_train, "POLY")
getAccuracy(x_train, y_train, svm_multi) 
getAccuracy(x_test, y_test, svm_multi) 

In [63]:
svm_multi1 = trainModel(x_train,y_train, "RBF")
getAccuracy(x_train, y_train, svm_multi1) 
getAccuracy(x_test, y_test, svm_multi1) 

Accuracy 0.7454545454545455
F1 0.7454048578374995
Accuracy 0.6714285714285714
F1 0.669492360074227


In [81]:
type(y_train[0][0])

numpy.int32

In [90]:
def load_self_data(num_feat):
    files = glob.glob("./self-data/*")
    img = []
    for fi in files:
        img.append(cv2.imread(fi))
        
#     print(img_arr.shape)
    img_arr = np.array(img)
    y_label = [0,3,2,1] 
    
    f = [None]*7
    for i in range(img_arr.shape[0]):
        pic = img_arr[i]
        pic = pic.reshape((1,256,256,3))
        f[0],f[1],f[2] = FeatureGetColours(pic)
        f[3] = FeatureHaveDominantLine(pic)
        f[4],f[5],f[6] = FeatureHaveHorizontalVerticalDiagLine(pic)
        x = np.array(f).reshape((1,7)).astype(np.float32)
        y = np.array(i).reshape((1,1)).astype(np.int32)
#         getAccuracy(x, y, svm_multi1)
        _, response = svm_multi1.predict(x)
        print(response)
    
    return img_arr

own_img = load_self_data(7)

[[0.]]
[[3.]]
[[1.]]
[[1.]]


### Multi-modal model (first 4 features)

In [85]:
# catergories = ["bridge","coast","mountain","rainforest"]
# extracted_data = []
# testing_data = []
# for c in catergories:
#     extracted_data.append(load_data(c,"Training"))
#     testing_data.append(load_data(c,"Testing"))

x_train,y_train = prepareInput_comb(extracted_data)
x_test,y_test = prepareInput_comb(testing_data)
svm_multi = trainModel(x_train,y_train, "LINEAR")
getAccuracy(x_train, y_train, svm_multi) 
getAccuracy(x_test, y_test, svm_multi) 

Accuracy 0.6272727272727273
F1 0.6226615671295629
Accuracy 0.6142857142857143
F1 0.6107979126108588


In [ ]:
svm_multi = trainModel(x_train,y_train, "POLY")
getAccuracy(x_train, y_train, svm_multi) 
getAccuracy(x_test, y_test, svm_multi) 

In [86]:
svm_multi2 = trainModel(x_train,y_train, "RBF")
getAccuracy(x_train, y_train, svm_multi2) 
getAccuracy(x_test, y_test, svm_multi2) 

Accuracy 0.6651515151515152
F1 0.6639676306121294
Accuracy 0.6642857142857143
F1 0.6637663740245262


In [89]:
def load_self_data(num_feat):
    files = glob.glob("./self-data/*")
    img = []
    for fi in files:
        img.append(cv2.imread(fi))
        
#     print(img_arr.shape)
    img_arr = np.array(img)
    
    y_label = [0,3,2,1] 
    f = [None]*4
    for i in range(img_arr.shape[0]):
        pic = img_arr[i]
        pic = pic.reshape((1,256,256,3))
        f[0],f[1],f[2] = FeatureGetColours(pic)
        f[3] = FeatureHaveDominantLine(pic)
#         f[4],f[5],f[6] = FeatureHaveHorizontalVerticalDiagLine(pic)
        x = np.array(f).reshape((1,4)).astype(np.float32)
        y = np.array(y_label[i]).reshape((1,1)).astype(np.int32)
#         getAccuracy(x, y, svm_multi2)
        _, response = svm_multi2.predict(x)
        print(response)
    
    return img_arr

own_img = load_self_data(4)

[[1.]]
[[3.]]
[[2.]]
[[1.]]


### Single-modal model

In [ ]:
for feature in range(0,7):
    print("\n Training with feature", feature)
    x_train,y_train = prepareInput_single(extracted_data,feature)
    x_test,y_test = prepareInput_single(testing_data,feature)
    print("\n Linear")
    svm_multi = trainModel(x_train,y_train,"LINEAR")
    getAccuracy(x_train, y_train, svm_multi) 
    getAccuracy(x_test, y_test, svm_multi) 
    print("\n POLY")
    svm_multi = trainModel(x_train,y_train,"POLY")
    getAccuracy(x_train, y_train, svm_multi) 
    getAccuracy(x_test, y_test, svm_multi) 
    print("\n RBF")
    svm_multi = trainModel(x_train,y_train,"RBF")
    getAccuracy(x_train, y_train, svm_multi) 
    getAccuracy(x_test, y_test, svm_multi) 

In [ ]:
x_train,y_train = prepareInput_single(extracted_data,4)
f0 = x_train
f1 = [0]*len(f0)

fig,ax = plt.subplots(figsize=(6,6))
for cat in range(4):
    catergories = ["bridge","coast","mountain","rainforest"]
    i = (y_train == cat)
    ax.yaxis.set_ticks(np.arange(0,4,1))
    ax.scatter(f0, y_train, color= "green")

plt.yticks([0, 1, 2, 3], ["bridge","coast","mountain","rainforest"])
ax.set_xlabel("Feature 4: Number of Horizontal Lines / Total Number of Lines")
ax.set_ylabel("Classes")
legend = ax.legend(loc='upper left', bbox_to_anchor=(1.1, 1))

fig.savefig("linear-data2.jpeg", bbox_inches='tight')

In [ ]:
# x_train,y_train = prepareInput_single(extracted_data,5)
f0 = x_train
f1 = [0]*len(f0)

fig,ax = plt.subplots(figsize=(6,6))
for cat in range(4):
    catergories = ["bridge","coast","mountain","rainforest"]
    i = (y_train == cat)
    ax.yaxis.set_ticks(np.arange(0,4,1))
    ax.scatter(f0, y_train, color = "blue")

plt.yticks([0, 1, 2, 3], ["bridge","coast","mountain","rainforest"])
ax.set_xlabel("Feature 5: Number of Vertical Lines / Total Number of Lines")
ax.set_ylabel("Classes")
legend = ax.legend(loc='upper left', bbox_to_anchor=(1.1, 1))

fig.savefig("linear-data2.jpeg", bbox_inches='tight')

In [ ]:
x_train,y_train = prepareInput_single(extracted_data,6)
f0 = x_train
f1 = [0]*len(f0)

fig,ax = plt.subplots(figsize=(6,6))
for cat in range(4):
    catergories = ["bridge","coast","mountain","rainforest"]
    i = (y_train == cat)
    ax.yaxis.set_ticks(np.arange(0,4,1))
    ax.scatter(f0, y_train, color = "grey")

plt.yticks([0, 1, 2, 3], ["bridge","coast","mountain","rainforest"])
ax.set_xlabel("Feature 6: Number of Diagonal Lines / Total Number of Lines")
ax.set_ylabel("Classes")
legend = ax.legend(loc='upper left', bbox_to_anchor=(1.1, 1))

fig.savefig("linear-data2.jpeg", bbox_inches='tight')

### Extension: CNN Model

In [ ]:
import sklearn
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Dropout, Conv3D,MaxPooling3D, Conv2D, MaxPooling2D, Dense, Reshape,Flatten
from functools import partial

np.random.seed(42)
tf.random.set_seed(42)

In [ ]:
def load_data_small(arg, testOrtrain):
    files = glob.glob("./Data/" + testOrtrain + " set/" + arg + "/*")
    img_arr = []
    for f in files:
        i = cv2.imread(f)
        height, width, channel = i.shape[:3]
        i = cv2.resize(i,None,fx=0.5, fy=0.5, interpolation = cv2.INTER_CUBIC)
        img_arr.append(i)
        
    return np.array(img_arr)

In [ ]:
catergories = ["bridge","coast","mountain","rainforest"]
extracted_data = []
val_data = []
testing_data = []
for c in catergories:
    all_training = load_data_small(c,"Training")
    sz = int(all_training.shape[0]*0.8)   
    extracted_data.append(all_training[:sz])
    val_data.append(all_training[sz:])
    testing_data.append(load_data_small(c,"Testing"))
    

In [ ]:
def formatCNNInput(data):
    x = np.array(data)
    num_catergories = x.shape[1]
    x_train = x.reshape((x.shape[0]*x.shape[1], x.shape[2], x.shape[3], x.shape[4], 1))

    y = []
    for i in range(4):
        y.append(np.tile(i,(num_catergories,1)))
    y_train = np.vstack((y[0],y[1],y[2],y[3]))
    
    return x_train, y_train

x_train, y_train = formatCNNInput(extracted_data)
x_val, y_val = formatCNNInput(val_data)
x_test, y_test = formatCNNInput(testing_data)

In [ ]:
model = keras.models.Sequential([
    Conv3D(filters=32, kernel_size=(4,4,1), input_shape=(128,128, 3, 1)),
    keras.layers.MaxPooling3D(pool_size=(2,2,3)),
    Reshape((62,62,32)),
    Conv2D(filters=8, kernel_size=12),
    keras.layers.MaxPooling2D(pool_size=2),
    Conv2D(filters=8, kernel_size=4),
    keras.layers.MaxPooling2D(pool_size=2),
    Conv2D(filters=4, kernel_size=4),
    keras.layers.MaxPooling2D(pool_size=2),
    keras.layers.Flatten(),
    keras.layers.Dense(units=128, activation='relu'),
    keras.layers.Dropout(0.2),
    keras.layers.Dense(units=64, activation='relu'),
    keras.layers.Dropout(0.2),
    keras.layers.Dense(units=4, activation='softmax'),
])

In [ ]:
checkpoint_path = "Trained_Model/training_1/cp.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                 save_weights_only=True,
                                                 verbose=1,
                                                 save_best_only=True)

In [ ]:
optim = tf.keras.optimizers.Nadam(
    learning_rate=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-07,
    name='Nadam'
)

model.compile(
    loss="sparse_categorical_crossentropy", 
    optimizer=optim,
    metrics=["accuracy"])

In [ ]:
model.summary()

In [ ]:
with tf.device('/device:GPU:0'):
    history = model.fit(x_train,
                        y_train,
                        batch_size=64,
                        epochs=30,
                        validation_data = (x_val, y_val),
                        callbacks = [cp_callback])

In [ ]:
# summarize history for accuracy
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.show()

In [ ]:
# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.show()

In [ ]:
score = model.evaluate(x_test, y_test)

In [ ]:
model.save_weights("Trained_Model/cnn_classifier3/cnn")